<a href="https://colab.research.google.com/github/krishnakumar51/Fake-News-Detection-Model/blob/main/LSTM_%2B_Bidirectional_LSTM_Fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
# data preprocessing
df = df.dropna()

In [ ]:
X = df.drop('label', axis=1)
X.shape

(18285, 4)

In [ ]:
y= df['label']
y.shape

(18285,)

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
from keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.preprocessing.text import one_hot
from keras.layers import LSTM, Dense

In [ ]:
voc_size=5100

**One Hot Representation**

In [ ]:
messages= X.copy()

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# making the sentences ready for embeddig, like
# remving stopwords, lowering the case(for all),spliting n joining wherever required
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
corpus=[]
for i in range(0, len(messages)):
  print(i)
  review= re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [ ]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [ ]:
onehot_rep = [one_hot(words, voc_size) for words in corpus]
onehot_rep

[[622, 4137, 2454, 4799, 929, 4456, 3711, 1930, 3708, 4889],
 [3841, 900, 2850, 4014, 3885, 1131, 4760],
 [2211, 5025, 3674, 2860],
 [2808, 616, 267, 2785, 4586, 2319],
 [2759, 3885, 3017, 5056, 1424, 737, 3885, 3425, 3849, 982],
 [1713,
  4029,
  3724,
  3215,
  405,
  2389,
  1595,
  2615,
  3151,
  2636,
  1362,
  3957,
  2804,
  275,
  4760],
 [1822, 18, 4409, 4755, 2294, 2793, 4798, 3039, 2484, 2699, 3893],
 [2236, 2985, 4672, 3114, 616, 3883, 2389, 642, 2484, 2699, 3893],
 [1769, 3217, 2531, 984, 1720, 4229, 5043, 2060, 2389, 3935],
 [2038, 2940, 431, 1224, 2842, 1513, 2275, 1941],
 [2505, 3578, 412, 1664, 2485, 4250, 3246, 4922, 2513, 2018, 1172],
 [2785, 1463, 929, 4229, 2389, 616],
 [51, 2870, 3734, 3481, 2774, 4231, 1934, 4036, 4342],
 [4797, 2582, 4606, 3827, 406, 482, 3100, 2484, 2699, 3893],
 [4521, 4786, 4066, 2298, 1682, 2484, 2699, 3893],
 [2262, 2664, 4375, 2641, 1899, 3656, 3527, 4606, 4809, 3862],
 [3180, 3014, 900],
 [4151, 1710, 3114, 3324, 2389, 123, 181, 4760],
 

**Embedding**

In [ ]:
length = 20
embedded_docs = pad_sequences(onehot_rep, padding='pre', maxlen= length)
embedded_docs

array([[   0,    0,    0, ..., 1930, 3708, 4889],
       [   0,    0,    0, ..., 3885, 1131, 4760],
       [   0,    0,    0, ..., 5025, 3674, 2860],
       ...,
       [   0,    0,    0, ..., 2484, 2699, 3893],
       [   0,    0,    0, ..., 4919, 2239, 3161],
       [   0,    0,    0, ..., 1279, 3076, 1896]], dtype=int32)

In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  622,
       4137, 2454, 4799,  929, 4456, 3711, 1930, 3708, 4889], dtype=int32)

In [ ]:
# /creating models
embedding_vector_features=40
model= Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=length))
# dropout layer could be added
model.add(LSTM(100))
# dropout layer could be added
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            204000    
                                                                 
 lstm_2 (LSTM)               (None, 100)               56400     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 260,501
Trainable params: 260,501
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# for making the same model into bidirectional LSTM,
# just add birectional infront of LSTM layer.. also if test dataset is new and in large number use dropouts too
from keras.layers import Bidirectional

embedding_vector_features=40
model= Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=length))
# dropout layer could be added
model.add(Bidirectional(LSTM(100)))
# dropout layer could be added
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
x_final = np.array(embedded_docs)
y_final = np.array(y)

In [ ]:
x_final.shape, y_final.shape

((18285, 20), (18285,))

In [ ]:
from sklearn.model_selection import train_test_split as ttt
x_train,x_test, y_train,y_test = ttt(x_final,y_final, test_size=0.25, random_state=101)

**Model Traning**

In [ ]:
model.fit(x_train, y_train, validation_data=(x_test, y_test),epochs=10, batch_size=64)

Epoch 1/10
215/215 [==============================] - 12s 54ms/step - loss: 0.0047 - accuracy: 0.9989 - val_loss: 0.6792 - val_accuracy: 0.9081
Epoch 2/10
215/215 [==============================] - 10s 48ms/step - loss: 0.0066 - accuracy: 0.9981 - val_loss: 0.5441 - val_accuracy: 0.9158
Epoch 3/10
215/215 [==============================] - 10s 46ms/step - loss: 0.0049 - accuracy: 0.9985 - val_loss: 0.6839 - val_accuracy: 0.9040
Epoch 4/10
215/215 [==============================] - 8s 38ms/step - loss: 0.0033 - accuracy: 0.9993 - val_loss: 0.5513 - val_accuracy: 0.9182
Epoch 5/10
215/215 [==============================] - 10s 49ms/step - loss: 0.0073 - accuracy: 0.9977 - val_loss: 0.6194 - val_accuracy: 0.9154
Epoch 6/10
215/215 [==============================] - 10s 48ms/step - loss: 0.0034 - accuracy: 0.9985 - val_loss: 0.5411 - val_accuracy: 0.9127
Epoch 7/10
215/215 [==============================] - 9s 40ms/step - loss: 0.0014 - accuracy: 0.9996 - val_loss: 0.6991 - val_accuracy: 0

**Performance metrics and accuracy**

In [ ]:
y_pred = (model.predict(x_test))>0.5

143/143 [==============================] - 1s 7ms/step


In [ ]:
from sklearn.metrics import confusion_matrix as cm
cm(y_test,y_pred)

array([[2362,  196],
       [ 194, 1820]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9146981627296588